## 5. Longest Palindromic Substring
- Description:
  <blockquote>
    Given a string s, return the longest Palindromic Substring in s.
    **Example 1:**

  ```
  Input: s = "babad"
  Output: "bab"
  Explanation: "aba" is also a valid answer.

  ```

  **Example 2:**

  ```
  Input: s = "cbbd"
  Output: "bb"

  ```

  **Constraints:**

  -   `1 <= s.length <= 1000`
  -   `s` consist of only digits and English letters.
  </blockquote>

- URL: [Problem_URL](https://leetcode.com/problems/longest-palindromic-substring/description/)

- Topics: DP, Palindrome

- Difficulty: Medium

- Resources: example_resource_URL

### Solution 1, Expand From Centers using nonlocal to update the start/left index and max palindrome substring length

A palindrome mirrors around its center. So, for each possible center in the string, expand outward as long as the characters on both sides match.

Key Observations:

    Odd-length palindromes have a single character as the center (e.g., "aba" → center at 'b').
    Even-length palindromes have two characters as the center (e.g., "abba" → center between the two 'b's).

Thus, for a string of length n, there are 2n - 1 possible centers:

    n centers for odd-length palindromes (each character)
    n - 1 centers for even-length palindromes (between each pair of adjacent characters)


- Time Complexity: O(N^2)
  - There are 2n−1=O(n) centers. For each center, we call expand, which costs up to O(n).
  - Although the time complexity is the same as in the DP approach, the average/practical runtime of the algorithm is much faster. This is because most centers will not produce long palindromes, so most of the O(n) calls to expand will cost far less than n iterations.
  - The worst case scenario is when every character in the string is the same.
- Space Complexity: O(1)
  - We don't use any extra space other than a few integers. This is a big improvement on the DP approach.

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        if not s:
            return ""
    
        start = 0
        max_len = 1  # Every single char is a palindrome of length 1

        def expand(left: int, right: int):
            nonlocal start, max_len
            while left >= 0 and right < len(s) and s[left] == s[right]:
                current_len = right - left + 1
                if current_len > max_len:
                    start = left
                    max_len = current_len
                left -= 1
                right += 1

        for i in range(len(s)):
            # Odd-length palindromes (center at i)
            expand(i, i)
            # Even-length palindromes (center between i and i+1)
            expand(i, i + 1)

        return s[start:start + max_len]

### Solution 2, Expand From Centers, expand returns left and right indexes of the palindrome substring

This is often cleaner and more functional in style.
This version is often preferred because:

    It’s easier to test expand in isolation.
    It avoids side effects.
    It aligns with functional programming principles

- Time Complexity: O(N^2)
- Space Complexity: O(1)

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        if not s:
            return ""
        
        start = 0
        max_len = 1

        def expand(left: int, right: int) -> tuple[int, int]:
            # Expand as long as characters match
            while left >= 0 and right < len(s) and s[left] == s[right]:
                left -= 1
                right += 1
            # After loop, [left+1, right-1] is the valid palindrome
            return left + 1, right - 1

        for i in range(len(s)):
            # Odd-length
            l1, r1 = expand(i, i)
            # Even-length
            l2, r2 = expand(i, i + 1)

            # Update global best if needed
            len1 = r1 - l1 + 1
            len2 = r2 - l2 + 1
            if len1 > max_len:
                start, max_len = l1, len1
            if len2 > max_len:
                start, max_len = l2, len2

        return s[start:start + max_len]

### Solution 3, Expand From Centers, expand returns length of max palindrome substring length

This is a popular interview implementation because it’s concise and avoids mutable state in closures.

 How we compute start:

    For a palindrome of odd length L centered at i:
    → it spans from i - (L//2) to i + (L//2)
    → so start = i - (L - 1) // 2
    For a palindrome of even length L centered between i and i+1:
    → the center is at i + 0.5, and the leftmost index is i - (L//2 - 1) = i - (L - 2)/2
    → but note: (L - 1) // 2 still works! Because for even L, (L - 1)//2 == L//2 - 1

So the formula start = i - (max_len - 1) // 2 works for both cases!

Benefits:

    expand is purely functional (no side effects, just returns an int).
    Minimal data passed back.
    Math is clean once you see the symmetry.

- Time Complexity: O(N^2)
- Space Complexity: O(1)

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        if not s:
            return ""
        
        start = 0
        max_len = 1

        def expand(left: int, right: int) -> int:
            while left >= 0 and right < len(s) and s[left] == s[right]:
                left -= 1
                right += 1
            # Length of palindrome is (right - left - 1)
            return right - left - 1

        for i in range(len(s)):
            len1 = expand(i, i)       # Odd-length
            len2 = expand(i, i + 1)   # Even-length
            current_max = max(len1, len2)

            if current_max > max_len:
                max_len = current_max
                # Compute starting index of the palindrome
                start = i - (current_max - 1) // 2

        return s[start:start + max_len]

### Solution 4, Dynamic Programming
Solution description
- Time Complexity: O(N^2)
  - We declare an n * n table dp, which takes O(n2) time. We then populate O(n2) states i, j - each state takes O(1) time to compute.
- Space Complexity: O(N^2)
  - The table dp takes O(n^2) space.

In [ ]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        n = len(s)
        # where dp[i][j] = true means s[i...j] is a palindrome.
        # Only the upper triangle (i <= j) matters—substrings go forward.
        dp = [[False] * n for _ in range(n)]
        ans = [0, 0]

        # Every single characters is a palindrome
        for i in range(n):
            dp[i][i] = True

        # Check all adjacent pairs palindromes, strings of length 2
        for i in range(n - 1):
            if s[i] == s[i + 1]:
                dp[i][i + 1] = True
                ans = [i, i + 1]

        # Finding palindromes for lengths from 3 to n
        # We fill the table by increasing substring length, so when we compute dp[i][j], the smaller subproblem dp[i+1][j-1] has already been solved.
        for diff in range(2, n):
            for i in range(n - diff):
                j = i + diff
                # If the two ends match and the inside (from i+1 to j-1) is already known to be a palindrome, then this whole substring is a palindrome.”
                if s[i] == s[j] and dp[i + 1][j - 1]:
                    dp[i][j] = True
                    ans = [i, j]

        i, j = ans
        return s[i : j + 1]